In [1]:
import mne
import os
import numpy as np
import matplotlib.pyplot as plt
import timeit

In [2]:
# Project directories
project_dir = os.getcwd()
os.chdir(project_dir)
mne.set_config('SUBJECTS_DIR',os.path.join(project_dir,'recons'))
data_dir = os.path.join(project_dir, 'rawMEGdata')
rawfile = os.path.join(data_dir,'subj04NN_sess01_raw_tsss.fif')
results_dir = os.path.join(project_dir,'results')

In [3]:
# Read epochs
epochs = mne.read_epochs('data_tmp\\subj_{}-epo.fif'.format(0), preload=True)

Reading f:\MYPROJECTS16\project_demoMNEpython\data_tmp\subj_0-epo.fif ...
    Read a total of 2 projection items:
        EOG-planar--0.200-0.200-PCA-01 (1 x 204) active
        EOG-axial--0.200-0.200-PCA-01 (1 x 102) active
    Found the data of interest:
        t =    -200.00 ...     796.00 ms
        0 CTF compensation matrices available
Not setting metadata
418 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 2)
2 projection items activated


In [4]:
# Read noise covariance
noise_cov = mne.read_cov('data_tmp\\subj_{}-noise-cov.fif'.format(0))

    306 x 306 full covariance (kind = 1) found.
    Read a total of 2 projection items:
        EOG-planar--0.200-0.200-PCA-01 (1 x 204) active
        EOG-axial--0.200-0.200-PCA-01 (1 x 102) active


In [5]:
# Setup source space (computes a downsampled surface on white matter)
subjectname = 'subj04NN'
src = mne.setup_source_space(subject=subjectname, spacing='oct6', add_dist='patch')
# mne.viz.plot_bem(subject=subjectname, brain_surfaces='white', src=src,orientation='coronal')
mne.write_source_spaces('data_tmp\\subj_{}-src.fif'.format(0), src, overwrite=True)


Setting up the source space with the following parameters:

SUBJECTS_DIR = f:\MYPROJECTS16\project_demoMNEpython\recons
Subject      = subj04NN
Surface      = white
Octahedron subdivision grade 6

>>> 1. Creating the source space...

Doing the octahedral vertex picking...
Loading f:\MYPROJECTS16\project_demoMNEpython\recons\subj04NN\surf\lh.white...
Mapping lh subj04NN -> oct (6) ...
    Triangle neighbors and vertex normals...
Loading geometry from f:\MYPROJECTS16\project_demoMNEpython\recons\subj04NN\surf\lh.sphere...
Setting up the triangulation for the decimated surface...
loaded lh.white 4098/132113 selected to source space (oct = 6)

Loading f:\MYPROJECTS16\project_demoMNEpython\recons\subj04NN\surf\rh.white...
Mapping rh subj04NN -> oct (6) ...
    Triangle neighbors and vertex normals...
Loading geometry from f:\MYPROJECTS16\project_demoMNEpython\recons\subj04NN\surf\rh.sphere...
Setting up the triangulation for the decimated surface...
loaded rh.white 4098/135176 selected to s

In [6]:
# Link to tranformation file (registration of MEG - MRI anatomy)
trans = os.path.join(os.path.join(data_dir,subjectname + '-trans.fif'))

In [7]:
# Visualize source space
fig = mne.viz.plot_alignment(trans=trans, subject=subjectname, surfaces='white', coord_frame='head', src=src)
mne.viz.set_3d_view(fig, azimuth=173.78, elevation=101.75,
                    distance=0.30, focalpoint=(-0.03, 0.02, 0.05))

Using pyvistaqt 3d backend.



In [8]:
# Compute forward solution
conductivity = (0.3,)  # for single layer
# conductivity = (0.3, 0.006, 0.3)  # for three layers for EEG
model = mne.make_bem_model(subject=subjectname, ico=5, conductivity=conductivity)
bem = mne.make_bem_solution(model)

Creating the BEM geometry...
Going from 5th to 5th subdivision of an icosahedron (n_tri: 20480 -> 20480)
inner skull CM is  -2.08 -21.79  -3.43 mm
Surfaces passed the basic topology checks.
Complete.

Homogeneous model surface loaded.
Computing the linear collocation solution...
    Matrix coefficients...
        inner skull (10242) -> inner skull (10242) ...
    Inverting the coefficient matrix...
Solution ready.
BEM geometry computations complete.


In [9]:
# Forward model
fwd = mne.make_forward_solution(epochs.info, trans=trans, src=src, bem=bem,
                                        meg=True, eeg=False, mindist=5.0, n_jobs=1)
mne.write_forward_solution(os.path.join(project_dir,'data_tmp\\subj_{}-fwd.fif'.format(0)), fwd, overwrite=True)

Source space          : <SourceSpaces: [<surface (lh), n_vertices=132113, n_used=4098>, <surface (rh), n_vertices=135176, n_used=4098>] MRI (surface RAS) coords, subject 'subj04NN', ~24.7 MB>
MRI -> head transform : f:\MYPROJECTS16\project_demoMNEpython\rawMEGdata\subj04NN-trans.fif
Measurement data      : instance of Info
Conductor model   : instance of ConductorModel
Accurate field computations
Do computations in head coordinates
Free source orientations

Read 2 source spaces a total of 8196 active source locations

Coordinate transformation: MRI (surface RAS) -> head
    0.998123 0.048752 -0.037063      -0.37 mm
    -0.018625 0.818178 0.574663      31.55 mm
    0.058340 -0.572894 0.817550      50.91 mm
    0.000000 0.000000 0.000000       1.00

Read 306 MEG channels from info
105 coil definitions read
Coordinate transformation: MEG device -> head
    0.998461 -0.052463 -0.017983      -2.21 mm
    0.054980 0.978883 0.196886      10.23 mm
    0.007274 -0.197571 0.980261      52.70 mm


In [11]:
# Inverse operator
# notice, loose=0 means orientation constrain normal to the cortex; loose non-zero allows the orientation to vary
inv = mne.minimum_norm.make_inverse_operator(epochs.info, fwd, noise_cov, loose=0.2, depth=0.8)
mne.minimum_norm.write_inverse_operator('data_tmp\\subj{}-inv.fif'.format(0), inv, overwrite=True)

Converting forward solution to surface orientation
    Average patch normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Computing inverse operator with 306 channels.
    306 out of 306 channels remain after picking
Selected 306 channels
Creating the depth weighting matrix...
    204 planar channels
    limit = 7793/8195 = 10.003567
    scale = 4.60382e-08 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 2)
Computing rank from covariance with rank=None
    Using tolerance 2.8e-13 (2.2e-16 eps * 306 dim * 4.1  max singular value)
    Estimated rank (mag + grad): 304
    MEG: rank 304 computed from 306 data channels with 2 projectors
    Setting small MEG eigenvalues to zero (without PCA)


C:\Users\pantazis\AppData\Local\Temp\ipykernel_53476\2687742720.py:3: RuntimeWarning: Something went wrong in the data-driven estimation of the data rank as it exceeds the theoretical rank from the info (304 > 69). Consider setting rank to "auto" or setting it explicitly as an integer.
  inv = mne.minimum_norm.make_inverse_operator(epochs.info, fwd, noise_cov, loose=0.2, depth=0.8)


Creating the source covariance matrix
Adjusting source covariance matrix.


C:\Users\pantazis\AppData\Local\Temp\ipykernel_53476\2687742720.py:3: RuntimeWarning: Something went wrong in the data-driven estimation of the data rank as it exceeds the theoretical rank from the info (304 > 69). Consider setting rank to "auto" or setting it explicitly as an integer.
  inv = mne.minimum_norm.make_inverse_operator(epochs.info, fwd, noise_cov, loose=0.2, depth=0.8)


Computing SVD of whitened and weighted lead field matrix.
    largest singular value = 16.3661
    scaling factor to adjust the trace = 1.88017e+22 (nchan = 306 nzero = 2)
Overwriting existing file.
Write inverse operator decomposition in f:\MYPROJECTS16\project_demoMNEpython\data_tmp\subj0-inv.fif...
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written
    Writing inverse operator info...
    Writing noise covariance matrix.
    Writing source covariance matrix.
    Writing orientation priors.
    [done]
